<h1>Population generation</h1>

In [1]:
#necessary imports
import numpy as np
import pandas as pd
import scipy.stats
from numpy import random
from scipy.stats import truncnorm

In [2]:
#możliwe generowane pola. Generacja opiera się o przecięcie tego zbioru = Ω ze zbiorem generowanym = A ⊂ Ω
populationFields = ["age", "gender", "race", "BMI", "physicalActivity", "otherIllnesses"]

from enum import Enum
class OtherIllnesses(Enum):
    hypertension = 0
    asthma = 1
    cancer = 2
    diabetes = 3
    depression = 4
    angina = 5
    myocardialInfraction = 6
    irritableBowelSyndrome = 7
    stroke = 8
    migraine = 9
#end class


In [3]:
#każda funkcja zwraca ilość == amount danych dla całej populacji
def normal(amount, spec):
    return np.random.normal(spec[0], spec[1], amount)
def choice(amount, spec):
    return np.random.choice(spec[0], amount, True, spec[1])
def minmax (amount, spec):
    X = truncnorm((spec[0] - spec[2]) / spec[3], (spec[1] - spec[2]) / spec[3], loc=spec[2], scale=spec[3])
    return X.rvs(amount)


normal(10, [52, 12.5])
#normal(10, [27.3397, 4.77216])

array([42.11255233, 36.28569977, 50.39831414, 48.75391443, 41.3166839 ,
       37.41337583, 48.03079895, 44.89424744, 57.18274378, 44.94979368])

In [54]:
#przykładowe generowanie populacji z niektórymi cechami

testspec = ["age", "gender", "race", "BMI", "physicalActivity"]
testweights = [[56, 12.5],#age[mean, standard_deviation] - normal dist.
            [["f", "m"],[0.52, 0.48]],#gender[vals, weights] - weighted single choice
            [["white","black","other"],[0.66, 0.22, 0.12]],#race[vals, weights] - weighted single choice
              [12.1212, 44.8364, 27.3397, 4.77216 ], #BMI [min_val, max_val, mean, std]] -beta
              [0, 510, 127.095, 101.904]] #activityMinutes [min_val, max_val, mean, std]] -beta
            #otherIllnesses będziemy modelować z wykorzystaniem metody weighted single choice wybierając
            #ułamki wg występowania choroby w populacji
#raczej przekazanie funkcji zamiast stringów
testdistrmethods = [normal, choice, choice, minmax, minmax]
#testdistrmethods = [normal, choice, choice, normal]
testdata = [testspec, testweights, testdistrmethods]


0           [age, gender, race, BMI, physicalActivity]
1    [[56, 12.5], [[f, m], [0.52, 0.48]], [[white, ...
2    [<function normal at 0x0615E6A0>, <function ch...
dtype: object

In [86]:
class Generator :
    #metadata section
    author = ""
    title = ""
    link = ""
    population = 0
    positive_population = 0
    multimorbid = 0
    
    #generation spec values
    data = []
    
    def __init__(self, data, metadata):
        #save metadata
        if len(metadata) == 3 :
            author, title, link = metadata
        if len(metadata) == 4 :
            author, title, link, population = metadata
        if len(metadata) == 5 :
            author, title, link, population, positive_population = metadata    
        if len(metadata) == 6 :
            author, title, link, population, positive_population, multimorbid = metadata
        
        self.data = data
        return self
    
    def populateDatabase(amount, speclist):
        #przecięcie zboiru generowanych cech i wszystkich możliwych
        columns = set(speclist[0]).intersection(populationFields)

        df = pd.DataFrame()
        for index,vals,method in zip(speclist[0],speclist[1],speclist[2]) :
            df[index] = method(amount, vals)

        return df
    
populateDatabase(1500, data)

,age,gender,race,otherIllnesses
0,74.168026,f,white,asthma
1,68.652537,m,white,asthma
2,70.085278,m,white,hypertension
3,62.798409,m,white,hypertension
4,70.197420,m,white,hypertension
...,...,...,...,...
1495,71.019901,m,white,hypertension
1496,64.000206,f,white,asthma
1497,80.014508,m,white,hypertension
1498,84.080635,f,south asian,hypertension


In [50]:
#create a new article entry
import os.path


#metadata section
author = "Yogini V. Chudasama"
title = "Multimorbidity and SARS-CoV-2 infection in UK Biobank"
link = "https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7271848/"

#data section
population = 3002
positive_population = 1073
multimorbid = 329

spec = ["age", "gender", "race", "otherIllnesses"]
weights = [[70, 6.5],
           [["f", "m"],[0.46, 0.54]],
           [["white","south asian", "black","other"],[0.85, 0.05, 0.06, 0.04]],
           [["hypertension", "asthma", "cancer", "diabetes"],[0.78, 0.12, 0.01, 0.09]]]
#raczej przekazanie funkcji zamiast stringów
distrmethods = [normal, choice, choice, choice]
data = [spec, weights, distrmethods]


#autmoatically find next file name in namespace
i = 0
while print(os.path.isfile(f"./data/data{i}.csv")) == True :
    i+=1
#save dataframe to file
pd.DataFrame(data).to_csv(f"data/data{i}.csv")
#write a corresponding entry to metadata file
newrow = pd.Series(author, title, link, population, positive_population, multimorbid)
pd.read_csv("data/metadata.csv").append(newrow).to_csv("data/metadata.csv")

True
